In [1]:
import socket
import threading
import os

In [2]:
clients = []
ALLOWED_EXTENSIONS = ['.txt', '.jpg', '.pdf']
BANNED_WORDS = ['badword1', 'badword2']

In [3]:
s = socket.socket()

In [4]:
def broadcast(message, sender=None):
    for client in clients:
        if client != sender:
            client.send(message)

In [5]:
def handle_client(client, addr):
    print(f"Connected with {addr}")
    
    client.send("NICK".encode())
    nickname = client.recv(1024).decode()
    
    clients.append(client)
    
    print(f"Nickname is {nickname}")
    broadcast(f"{nickname} joined the chat!".encode())
    client.send("Connected to the server!".encode())
    
    while True:
        msg = client.recv(1024)
        
        if not msg:
            break
            
        decoded = msg.decode()
        lower_msg = decoded.lower()
        
        if decoded == "exit":
            broadcast(f"{nickname} left the chat!".encode(), client)
            clients.remove(client)
            client.close()
            break
        
        if decoded.startswith("FILE:"):
            _, filename, filesize = decoded.split(":")
            filesize = int(filesize)
        
            ext = os.path.splitext(filename)[1]
        
            if ext not in ALLOWED_EXTENSIONS:
                client.send(f"REJECTED:{ext}".encode())
                continue
        
            client.send("ACCEPTED".encode())
        
            file_data = b""
            bytes_read = 0
        
            while bytes_read < filesize:
                data = client.recv(1024)
                file_data += data
                bytes_read += len(data)
        
            for c in clients:
                if c != client:
                    c.send(f"FILE_FROM_SERVER:{filename}:{filesize}".encode())
        
                    c.sendall(file_data)
        
            print(f"{nickname} sent file {filename}")
        
        elif any(bad_word in lower_msg for bad_word in BANNED_WORDS):
            client.send("Server: Message contains inappropriate content!".encode()) 
        
        else:
            broadcast(msg, client)
            
    print(f"Disconnected with {addr}")

In [6]:
s.bind(('localhost', 9999))
s.listen(5)
print("Server started. Waiting for connections...")

Server started. Waiting for connections...


In [ ]:
while True:
    client, addr = s.accept()

    thread = threading.Thread(target=handle_client, args=(client, addr))
    thread.start()

Connected with ('127.0.0.1', 55327)
Nickname is c1
Connected with ('127.0.0.1', 55761)
Nickname is c2
Connected with ('127.0.0.1', 55823)
Nickname is c3
c1 sent file 1770918764658.jpg
Disconnected with ('127.0.0.1', 55327)
Disconnected with ('127.0.0.1', 55823)
Disconnected with ('127.0.0.1', 55761)
